# 02 - Item Quicklook Notebook
Lightweight playground to parameterize STAC searches and inspect asset metadata.

## Usage
1. Export `EDITO_API_TOKEN` (and optionally `EDITO_DATA_BASE_URL`).
2. Adjust the search parameters below.
3. Run the search cell, then inspect a specific item.

In [ ]:
import os
from pathlib import Path
from typing import Any, Dict, List, Optional

import pandas as pd
import requests

BASE_URL = os.getenv("EDITO_DATA_BASE_URL", "https://api.dive.edito.eu/data").rstrip("/")
API_TOKEN = os.getenv("EDITO_API_TOKEN")

session = requests.Session()
session.headers.update({"Accept": "application/json"})
if API_TOKEN:
    session.headers["Authorization"] = f"Bearer {API_TOKEN}"
else:
    print("Warning: EDITO_API_TOKEN not found; only public endpoints will succeed.")

BASE_URL


In [ ]:
def absolute_url(path: str) -> str:
    return path if path.startswith("http") else f"{BASE_URL}{path}"

def stac_search(collections: List[str], bbox: Optional[List[float]] = None, datetime_range: Optional[str] = None, limit: int = 10) -> Dict[str, Any]:
    payload: Dict[str, Any] = {"collections": collections, "limit": limit}
    if bbox:
        payload["bbox"] = bbox
    if datetime_range:
        payload["datetime"] = datetime_range
    response = session.post(absolute_url("/search"), json=payload, timeout=90)
    response.raise_for_status()
    return response.json()

def items_to_frame(features: List[Dict[str, Any]]) -> pd.DataFrame:
    rows = []
    for feat in features:
        props = feat.get("properties", {})
        assets = feat.get("assets", {})
        rows.append({
            "id": feat.get("id"),
            "collection": feat.get("collection"),
            "datetime": props.get("datetime"),
            "asset_keys": list(assets.keys()),
            "bbox": feat.get("bbox"),
        })
    return pd.DataFrame(rows)


In [ ]:
collection_filter = ["climate_forecast-sea_water_potential_temperature"]
bbox_filter = [-10.0, 40.0, 10.0, 60.0]
datetime_filter = "2024-10-01T00:00:00Z/2024-10-05T00:00:00Z"
limit = 20
search_response = stac_search(collection_filter, bbox_filter, datetime_filter, limit)
features = search_response.get("features", [])
items_df = items_to_frame(features)
items_df


In [ ]:
row_index = 0
if features and 0 <= row_index < len(features):
    selected = features[row_index]
    display({
        "id": selected.get("id"),
        "collection": selected.get("collection"),
        "datetime": selected.get("properties", {}).get("datetime"),
        "assets": selected.get("assets"),
    })
else:
    print("Row index out of bounds or no results.")
